In [ ]:
import os
import sys

cwd0 = './config/'
sys.path.append(cwd0)

import visualID_Eng as vID
from visualID_Eng import color, fg, bg, hl
vID.init(cwd0)
import tools4pyPhysChem as t4pPC

#cancel the "last operation show-up" specific of Jupyter notebooks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

__author__ = "Iann C. Gerber & Romuald Poteau, LPCNO (INSA-CNRS-UT3 lab, Toulouse, France)"

# Parametrized tight-binding for calculating a material's energy and electronic structure

<figure>
<img width = "800px" src="MiscImages/Introductions/TB3.webp" alt="Image TB" />
<figcaption>Image made by R.P. with ChatGPT-4o (OpenAI, 11/2024)</figcaption>
</figure> 

## Introduction

### The tight-binding method

<div class="intro">
    
The tight binding (TB) method is an approximate approach for solving for the electronic wave functions for electrons in solids assuming a basis of localized atomic-like orbitals. This model is applicable to a wide variety of systems and phenomena in quantum physics. The approach does not require computing from first principles, but instead simply uses parameterized matrix elements. In contrast to *ab initio* or DFT calculations, the tight-binding model can scale to large system sizes on the order of millions of atoms. 

TB calculations proposed in this notebook will be done with [**pybinding**](https://docs.pybinding.site/en/stable/). It is a scientific Python package for numerical tight-binding calculations in solid state physics.

<figure>
<img width = "200px" src="MiscImages/TB/pb.webp" alt="pybinding logo" />
</figure> 

The orbitals are assumed to be orthonormal, pybinding focuses on the “empirical tight binding” approach in which the Hamiltonian matrix elements are simply parametrized.
</div>
<br>
<div class="rq">

This package actually does simple Hückel calculations, but with periodic boundary conditions. It can also do molecular calculations. The Hückel electron-hopping parameter, named $\beta$ in the chemistry realm, is often referred to as $t$ in solid-state physics, $t$ meaning *transfer*. It is also called the bond energy or two center integral and it is the dominant term in the tight binding model.
</div>

### The band structure of solids and the Fermi level

<div class="intro">

Before playing with a TB package, it is necessary to remind that in the [band theory of solids](https://en.wikipedia.org/wiki/Fermi_level), electrons are considered to occupy a series of bands composed of single-particle energy eigenstates each labelled by &varepsilon;. Although this single particle picture is an approximation, it greatly simplifies the understanding of electronic behaviour and it generally provides correct results when applied correctly.

The Fermi–Dirac distribution gives the probability that, at thermodynamic equilibrium, a state having energy &varepsilon; is occupied by an electron:

$$f(\epsilon)=\frac{1}{1+\exp\left(\frac{\epsilon-\mu}{k_{\mathrm{B}}T}\right)}$$

where *k*<sub>B</sub> is the Boltzmann constant, *T* is the absolute temperature and *&mu;* is the total chemical potential.

At finite temperatures there is no sharply defined most energetic electron because thermal energy is continuously exciting electrons within the bands. It is possible to define an energy level with an arbitrary 50% probability of occupation, and this is the Fermi level, sometimes called Fermi energy, whereas the actual Fermi energy is defined at *T*=0K. Some examples are reported below, for different materials. 

<figure>
  <img width = "600px" src="MiscImages/TB/Band_filling_diagram.svg" alt="band structure of solids" />
  <figcaption><b>Density of states and filling of the electronic states in various types of materials at equilibrium at <i>T</i> &ne; 0K</b><br>(from <a href="https://en.wikipedia.org/wiki/Fermi_level">Wikipedia</a>)</figcaption>
</figure>

</div>



## Doing a TB calculation with pybinding: workflow

<div class="intro">

The general workflow starts with the definition of a model. The crystal lattice and the system geometry are the two main main parts required to describe a tight-binding model.

Once the model description is complete, pybinding will build the tight-binding Hamiltonian matrix. The next step is to apply computations to the matrix to obtain the values of the desired quantum properties. To that end, there are the following possibilities: Kernel polynomial method, exact diagonalization, or a user-defined solver

After the main computation is complete, various utility functions are available for post-processing the raw result data. The included plotting functions are tailored for tight-binding problems to help visualize the model structure and to make sense of the results

</div>

### Lattice and geometry specification

<div class="introT" title="The crystal lattice">

This step includes the specification of the primitive lattice vectors and the configuration of the unit cell (atoms, orbitals, TB parameters and spins). This can be user-defined, but the package also contains a repository of the pre-made specifications for several materials.
</div>
<br>
<div class="introT" title="System geometry">

The model system can be infinite through the use of translational symmetry or it can be finite by specifying a shape. 
</div>

<div class="rqT" title="first install the development version of the pybinding package">

It can be done with pip. Uncomment the next line the first time you use this notebook
</div>

In [ ]:
# %pip install pybinding-dev

In [ ]:
import pybinding as pb
import numpy as np
import matplotlib.pyplot as plt

pb.pltutils.use_style() # applies pybinding’s default style settings for matplotlib

d = 0.1  # [nm] unit cell length
t = -1   # [eV] hopping energy

# create a simple 1D lattice with vector a1
lattice = pb.Lattice(a1=[d, 0], a2=[0, d])
lattice.add_sublattices(
    ('A', [0, 0], 0.)  # add an atom called 'A' at position [0, 0] and set its associated AO energy to 0 eV
)
lattice.add_hoppings(
    # (Difference of the indices of the source and destination unit cells
    # Name of the sublattice in the source unit cell, Name of the sublattice in the destination unit cell, hopping energy)
    ([0, 1], 'A', 'A', t),
    ([1, 0], 'A', 'A', t)
)

<div class="rq"'>
    
It may not be immediately obvious what this code does. Fortunately, [`Lattice`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Lattice.html#pybinding.Lattice) objects have a convenient [`Lattice.plot()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Lattice.html#pybinding.Lattice.plot) method to easily visualize the constructed lattice.
</div>

In [ ]:
lattice.plot()  # plot the lattice that was just constructed
plt.show()      # standard matplotlib show() function

<div class="rq">

In the figure we see lattice vectors $a_1$ and $a_2$ which were used to initialize [`Lattice`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Lattice.html#pybinding.Lattice). These vectors describe a Bravais lattice with an infinite set of positions,

$$
\vec{R} = n_1 \vec{a}_1 + n_2 \vec{a}_2,
$$

where $n_1$ and $n_2$ are integers. The blue circle labeled A represents the atom which was created with the [`Lattice.add_sublattices()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Lattice.html#pybinding.Lattice.add_sublattices) method. The slightly faded out circles represent translations of the lattice in the primitive vector directions, i.e. using the integer index $[n_1, n_2]$.

The hoppings are specified using the [`Lattice.add_hoppings()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Lattice.html#pybinding.Lattice.add_hoppings) method and each one consists of `(relative_index, from_sublattice, to_sublattice, energy)`:

* The main cell always has the index $[n_1, n_2]$ = [0, 0]. The `relative_index` represents the number of integer steps needed to reach another cell starting from the main one. Each cell is labeled with its `relative_index`, as seen in the figure.
  
* A hopping is created between the main cell and a neighboring cell specified by `relative_index`. Two hoppings are added in the definition: [0, 1] and [1, 0].

* The opposite hoppings [0, -1] and [-1, 0] are added automatically to maintain translational symmetry.
  
* This lattice consists of only one sublattice so the `from` and `to` sublattice fields are trivial. Generally, `from_sublattice` indicates the sublattice in the [0, 0] cell and `to_sublattice` in the neighboring cell. This will be explained further in the next example.
  
* The last parameter is simply the value of the hopping energy.
  
It’s good practice to build the lattice inside a function to make it easily reusable. Here we define the same lattice as before, but note that the unit cell length and hopping energy are function arguments, which makes the lattice easily configurable.
</div>
<br>
<div class="intro">

The defined square is the primitive of the tetragonal 2D [Bravais lattice](https://en.wikipedia.org/wiki/Bravais_lattice), see figure below (*a* = *b* and &theta; = 90°). Let's recall that a primitive cell is a unit cell that contains exactly one lattice point. 
<img width="150" src="./MiscImages/TB/SQL.svg" style="display:block; margin-left:auto; margin-right:auto" id="SQL"/> 

This is the case here, since each atom is shared by 4 cells, it is counted as 1/4. 4 atoms x 1/4 = 1.
 
</div>

<div class="intro">

It is even possible to plot the **[Brillouin zone](https://en.wikipedia.org/wiki/Brillouin_zone)**. The [reciprocal lattice vectors](https://en.wikipedia.org/wiki/Reciprocal_lattice) **b<sub>1</sub>** and **b<sub>2</sub>** are calculated automatically based on the real space vectors. There is no need to specify them manually. The first Brillouin zone is determined as the Wigner–Seitz cell in reciprocal space. By default, the plot method labels the vertices of the Brillouin zone.
</div>

In [ ]:
lattice.plot_brillouin_zone()

### Setting up a band structure calculation

<div class="intro">

In order to calculate the band structure of a crystal lattice, this section introduces the concepts of a [`Model`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Model.html#pybinding.Model) and a [`Solver`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.solver.html#pybinding.solver.Solver).

A [`Model`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Model.html#pybinding.Model) contains the full tight-binding description of the physical system that we wish to solve. We’ll now assign the previous lattice to the model.
</div>

#### Definition of the model

In [ ]:
model = pb.Model(
    lattice,  # predefined lattice from the material repository
)

model.hamiltonian
model.plot()

<div class="rq">

It’s a sparse matrix (see [`scipy.sparse.csr_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix)) which corresponds to the tight-binding Hamiltonian of our model. The output above shows the default sparse representation of the data where each line corresponds to `(row, col) value`. Alternatively, we can see the dense matrix output:
</div>

In [ ]:
model.hamiltonian.todense()

<div class="introT" title="Set up of an infinite 2D sheet">

Next, we include [`translational_symmetry()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.translational_symmetry.html#pybinding.translational_symmetry) to create an infinite graphene sheet.
</div>

In [ ]:
model = pb.Model(
    lattice,
    pb.translational_symmetry()
)
model.plot()

<div class="rq">

The red lines indicate hoppings on periodic boundaries. The lighter colored circles represent the translations of the unit cell. The number of translations is infinite, but the plot only presents the first one in each lattice vector direction.

In the figure above, notice that 4 translations of the unit cell are presented and it appears as if 4 atoms are missing. This is only in appearance. By default, `model.plot()` shows just the first-nearest translations of the unit cell
</div>

#### Solver

<div class="intro">

A [`Solver`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.solver.html#pybinding.solver.Solver) can exactly calculate the eigenvalues and eigenvectors of a Hamiltonian matrix. Although there are various [Eigenvalue solvers](http://docs.pybinding.site/page/tutorial/solvers.html) in pybinding, right now we’ll just use the [`lapack()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.solver.html#pybinding.solver.lapack) solver which is the simplest and most appropriate for small systems.

A great advantage of this solver is that it always solves for **all** eigenvalues and eigenvectors of a Hamiltonian matrix.
</div>

In [ ]:
solver = pb.solver.lapack(model)
eigenvalues = solver.calc_eigenvalues()
eigenvectors = solver.eigenvectors

eigenvalues
eigenvectors

eigenvalues.plot()

## Useful functions to enhance the capabilities of pybinding

In [ ]:
from pathlib import Path

saveFig_path = Path() / "FiguresCreatedInNotebooks/TB"
saveFig_path_dev = Path() / "FiguresCreatedInNotebooks/TB-dev"
saveFig_path.mkdir(parents=True, exist_ok=True)
saveFig_path_dev.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, IMAGES_PATH="./", tight_layout=True, fig_extension="png", resolution=300):
    '''
    saves a figure to a given subfolder, that will be created if it does not exist
    input:
        - fig_id = name of the figure, without an extension
        - IMAGES_PATH = pathway to the folder. Default: "."
        - tight_layout = boolean. Automatically adjusts subplot params so that the subplot(s) fits in to the figure area. Default: True
        - fig_extension = image type. Default: png
        - resolution = resolution of the figure, in dpi. Default: 300
    '''
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution, transparent=True)
    return

In [ ]:
def lorentzian(v, v0, I, w):
    import numpy as np
    """
    A lorentzian function that takes linewidth at half intensity (w) as a
    parameter.
    Input:
        - v: Array of values at which to evaluate distribution
        - v0: Center of the distribution (= position of the peak)
        - I: intensity of the peak
        - w: Peak width at half max intensity

    Returns: Lorentzian distribution evaluated at points in v
    """
    return I * ((w/2.0) / ((w/2.0)**2 + (v - v0)**2)) / np.pi

def gaussian(x, mu, I, w):
    """
    A gaussian function that takes linewidth at half intensity (sig) as a parameter.
    Input:
        - x: array of values at which to evaluate distribution
        - mu: center of the distribution (= position of the peak)
        - I: intensity of the peak
        - w: peak width at half max intensity

    Returns: Gaussian distribution evaluated at points in x
    """
    import numpy as np
    sig = w / (2.0 * np.sqrt(2.0*np.log(2.0))) 
    return I / (np.sqrt(2.0 * np.pi) * sig) * np.exp(-((x - mu) / sig)**2 / 2.0)

def sum_signals(linspace, peaklist, w, broadening):
    """
    Given a numpy linspace a spectrum as a list of (frequency, intensity)
    tuples, and a linewidth, returns an array of y coordinates for the
    lineshape.
    Input:
        - linspace: a numpy linspace of x coordinates for the lineshape.
        - peaklist: a list of (frequency, intensity) tuples
        - w: peak width at half maximum intensity
        - broadening (string): returns a 'Gaussian' distribution
                      otherwise returns a 'Lorentzian' distribution

    Returns: array of y coordinates for the lineshape
    """
    result = np.zeros(len(linspace))
    for p in peaklist:
        if broadening == 'Gaussian':
            result += gaussian(linspace, p, 1.0, w)
        elif broadening == 'Lorentzian':
            result += lorentzian(linspace, p, 1.0, w)
        else:
            print("No broadening???")
    return result

def norm(V):
    '''
    Calculates the norm of a vector
    Input:
        - V = numpy array with 3 values
    Returns:
        - the norm of V
    '''
    return np.sqrt(V[0]**2 + V[1]**2 + V[2]**2)

def MK(reciprocal_vectors,nmesh_k):

    normb1 = norm(reciprocal_vectors[0])
    nb1 = np.linspace(1, nmesh_k[0], nmesh_k[0])
    kb1 = (2*nb1 - nmesh_k[0] - 1) / nmesh_k[0]
    if (reciprocal_vectors.shape[0] == 1):
        kx_grid = normb1 * kb1
        normb2 = 0
        ky_grid = [0]
    else:
        normb2 = norm(reciprocal_vectors[1])
        nb2 = np.linspace(1, nmesh_k[1], nmesh_k[1])
        kb2 = (2*nb2 - nmesh_k[1] - 1) / nmesh_k[1]
        kx_grid = kb1 * reciprocal_vectors[0][0] + kb2 * reciprocal_vectors[1][0]
        ky_grid = kb1 * reciprocal_vectors[0][1] + kb2 * reciprocal_vectors[1][1]
    return kx_grid, ky_grid, normb1, normb2

def plot_kdosMP(solver, lattice, nmesh_k = (250,250), nmesh_E = 1000,
                broadening = 'Gaussian', HWHM = 0.2, saveFig_path=".",FigName=None):
    '''
    Calculates and then plots the density of states.
    First solves the model on a k-point mesh
    An then broadens the energy levels with a Gaussian or a Lorentzian function
    Input: 
        - solver = pybinding solver instance
        - lattice = pybinding lattice of the model
        - nmesh_k = tuple that defines the number of kpoints in each dimension
        - nmesh_E = resolution of the energy for the DOS plot
        - broadening = 'Gaussian' or 'Lorentzian'
        - HWHM = half-width at half-maximum of the Lorentzian or Gaussian broadening function
        - saveFig_path = pathway where to save the DOS plot 
        - Figname = name of the DOS graphic output file 
    Output:
        - DOS plot, E = f(DOS)
        - integrated DOS between -oo and +oo
    Saves :
        - a Figname.png file if this variable is not None (the default)
    '''

    from scipy.integrate import simpson
    import numpy as np

    reciprocal_vectors = np.array(lattice.reciprocal_vectors())

    if (reciprocal_vectors.shape[0] == 2):
        print(f"{fg.RED}{hl.BOLD}Please wait. The DOS calculation can take time for 2D systems{hl.OFF}")

    # kpoints. Monkhsor-Pack mesh
    kb1_grid, kb2_grid, normb1, normb2 = MK(reciprocal_vectors,nmesh_k)
    
    norm = np.array([normb1, normb2])
    for i, v in enumerate(reciprocal_vectors):
        print(f"b{i+1:1d} = {np.array2string(v, formatter={'float_kind':'{:8.2f}'.format})}. Norm = {norm[i]:.1f} nm-1")

    bands = []
    for kb1 in kb1_grid:
        for kb2 in kb2_grid:
            solver.set_wave_vector([kb1,kb2])
            bands.append(solver.eigenvalues)
    bands = np.array(bands)
    bands = np.sort(bands, axis=None)
    Emin = min(bands)
    Emax = max(bands)
    Egrid = np.linspace(Emin-1, Emax+1, nmesh_E)
    nkp = len(kb1_grid) * len(kb2_grid)
    print(f"Number of k-points = {nkp} ({nmesh_k[0]} x {nmesh_k[1]} grid)")   
    dos = sum_signals(Egrid,bands,HWHM,broadening="Gaussian")
    dos = dos / nkp
    max_dos = max(dos)

    fig, ax = plt.subplots()
    _ = ax.plot(dos, Egrid, lw = 2, color='#2c8586')
    _ = ax.set_xlabel("DOS")
    _ = ax.set_ylabel("E / eV")
    _ = ax.set_xlim(0.,max_dos*1.1)
    # _ = ax.set_xticks([])
    plt.tight_layout()
    if FigName is not None:
        save_fig(FigName, saveFig_path)
    plt.show()
    I = simpson(dos,x=Egrid)
    print(f"Integral of the DOS between -oo and +oo = {I:.0f} state(s)")
    return


<div class="rqT" title="Lorentzian shape vs Gaussian shape?">

Gaussian and Lorentzian functions play extremely important roles in science. their general mathematical expressions are given by:

$$\mathcal{G}(x)=\frac{1}{\sigma\sqrt{2\pi}}\exp\left(-\frac{(x-x_{0})^{2}}{2\sigma^{2}}\right)$$

$$\mathcal{L}(x)=\frac{1}{\pi}\frac{\frac{1}{2}w}{(x-x_{0})^{2}+\left(\frac{1}{2}w\right)^{2}}$$

where *x*<sub>0</sub> is the center of each function.
- *w* is a parameter that directly specifies the full width at half maximum (FWHM) of the $\mathcal{L}(x)$ function
- *&sigma;* in the $\mathcal{G}(x)$ expression is related to the FWHM value *w* by the relationship $w=2\sigma\sqrt{2\ln2}$
</div>

In [ ]:
peak = [0.0]
x = np.linspace(-3,3,3000)
width=0.05
yG = sum_signals(x, peak, width, 'Gaussian')
_ = plt.plot(x,yG,label=f"Gaussian. Width={width}")
yL = sum_signals(x, peak, width, 'Lorentzian')
_ = plt.plot(x,yL,label=f"Lorentzian. Width={width}")
_ = plt.legend()
_ = plt.xlim(-0.3,0.3)
_ = plt.show()
from scipy.integrate import simpson
print(f"Gaussian. Integral = {simpson(yG,x=x)}")
print(f"Lorentzian. Integral = {simpson(yL,x=x)}")


<div class="exE">

Plot the $\mathcal{G}(x)$ function with the parameters:
- *x*<sub>0</sub> = 3
- *w* = 1.0

Plot the $\mathcal{L}(x)$ function on the same graph, with:
- *x*<sub>0</sub> = 1
- *w* = 0.5

Calculate the integral of both functions. Comment on the result
</div>

In [ ]:
# insert your code here



<div class="sol">

Want to see a possible answer? Uncomment the `# %load ./SolutionsToExercises/...` command below, and then run the cell
</div>

In [ ]:
# %load ./SolutionsToExercises/TB/G_vs_L.py

## Infinite chain of equally spaced H atoms

### "Equilibrium" lattice parameters

#### Definition of the model

In [ ]:
import pybinding as pb
import numpy as np
import matplotlib.pyplot as plt

pb.pltutils.use_style() # applies pybinding’s default style settings for matplotlib

def Hchain_lattice(a=0.1,t=-1.0):
    # create a simple 1D lattice with vector a1
    lattice = pb.Lattice(a)
    lattice.add_one_sublattice('H', [0])
    lattice.add_one_hopping([1], 'H', 'H', t)
    return lattice
    
aH = 0.1   # [nm] unit cell length
tH = -1.0   # [eV] hopping energy

Hlattice = Hchain_lattice(aH,tH)

# single atom definition cannot be plotted
# Hlattice.plot()  # plot the lattice that was just constructed
# plt.show()      # standard matplotlib show() function
Hlattice.plot_brillouin_zone()
plt.show()

Hmodel = pb.Model(
    Hlattice,
    pb.translational_symmetry()
)
Hmodel.plot()
plt.show()

#### Diagonalization of the hamiltonian

In [ ]:
Hmodel.hamiltonian
Hsolver = pb.solver.lapack(Hmodel)
eigenvalues = Hsolver.calc_eigenvalues()
eigenvectors = Hsolver.eigenvectors

eigenvalues
eigenvectors

#### Band structure calculation and plot

In [ ]:
from math import pi

Gamma = [0, 0]
X = [pi/aH, 0]
kpoints_labels = [f'$\Gamma$', 'X']
Hbands = Hsolver.calc_bands(Gamma, X)

Hbands.plot_kpath(point_labels=kpoints_labels)

In [ ]:
Hbands.plot(point_labels=kpoints_labels)

<div class="rq">

This is exactly what can be found in textbooks, such as in the remarkable *Solids And Surfaces. A Chemist's View of Bonding in Extended Structures*, by R. Hoffmann (Wiley, 1988).
<figure>
  <img width = "40%" src="MiscImages/TB/Hoffmann-Hchain-p7.png" alt="H chain" />
  <figcaption >(from R. Hoffmann, <i>A Chemist's View of Bonding in Extended Structures</i>, &copy; Wiley 1991)</figcaption>
</figure>

This band is nothing else than the Fermi level, *i.e.* the HOMO. 
</div>

#### Density of states (DOS)

<div class="intro">

For analysis purposes, one often plots the total density of states *g*(*&epsilon;*), defined by:

$$g(\varepsilon)=\frac{2}{(2\pi)^{d}}\sum_{i}\int_{\mathrm{BZ}}\delta(\varepsilon-\varepsilon_{i,\mathbf{k}})d\mathbf{k}$$

Mathematically speaking, the **k**-point set is expressed by a set of fractional numbers *&kappa;*, the coordinates of the k-points in a coordinate system spanned by the reciprocal lattice vectors **b**<sub>*j*</sub>. For example, for a square lattice constant *a*, one chooses:

$$\boldsymbol{\mathrm{k}}=(\kappa_{x},\kappa_{y})\frac{\pi}{a}$$

There are several possibilities to define *&kappa;*, among them the Monkhorst and Pack. Monkhorst-Pack grids are used to approximate the value of an integral over the Brillouin zone by sampling reciprocal space on a regular grid of **k**-points<br>An *N<sub>x</sub>* &times; *N<sub>y</sub>* grid of **k**-points is calculated as:

$$\boldsymbol{\kappa}_{x}=\frac{2n_{x}-N_{x}-1}{2N_{x}}a,\,\,\,\,\,\,\,n_{x}=1,\ldots N_{x}$$

$$\boldsymbol{\kappa}_{y}=\frac{2n_{y}-N_{y}-1}{2N_{y}}a,\,\,\,\,\,\,\,n_{y}=1,\ldots N_{y}$$

with integers *n<sub>x</sub>* and *n<sub>y</sub>* ( the factor 2 in the denominators ensures that the **k**-points stay away from the coordinate axis of reciprocal space).<br>
It is the special Monkhorst-Pack **k**-point set that is implemented in the `plot_dos` function. 
</div>

In [ ]:
plot_kdosMP(Hsolver, Hlattice, saveFig_path=saveFig_path, FigName="DOS_H1D")

<div class="rqT" title="General definition of the Monkhorst-Pack mesh">

$$\boldsymbol{\mathcal{\mathrm{k}}=}\sum_{i=1}^{3}\frac{2n_{i}-N_{i}-1}{2N_{i}}\boldsymbol{\mathrm{b}}_{i},\,\,\,\,\,\,\,n_{i}=1,\ldots N_{i}$$
</div>

### Increase of the lattice parameter, band width

<div class="intro">

With a simple tight-binding model, it mainly consists in decreasing the hopping integral, *t*. The lattice paramer, *a*, is decreased as well for plotting purposes, but since *t* does not depend on *a*, changing this parameter has no impact.
</div>

In [ ]:
aH2 = 2*aH
tH2 = tH/2

Hlattice2 = Hchain_lattice(aH2,tH2)
Hmodel2 = pb.Model(
    Hlattice2,
    pb.translational_symmetry()
)
Hmodel2.plot()
plt.show()

Hsolver2 = pb.solver.lapack(Hmodel2)

Gamma = [0, 0]
X = [pi/aH2, 0]
Hbands2 = Hsolver2.calc_bands(Gamma, X)

Hbands2.plot(point_labels=kpoints_labels)
plt.show()

<div class="rq">

Compare it to the previous band structure, and to the next picture.
<figure>
  <img width = "30%" src="MiscImages/TB/Hoffmann-HchainBW-p8.png" alt="band width in a H chain" />
  <figcaption >(from R. Hoffmann, <i>A Chemist's View of Bonding in Extended Structures</i>, &copy; Wiley 1991)</figcaption>
</figure>

</div>

In [ ]:
plot_kdosMP(Hsolver2,Hlattice2, saveFig_path=saveFig_path, FigName="DOS_H1D_weakerInteraction")

### More than one AO in the unit cell

In [ ]:
aH2 = 2*aH
tH2 = tH

def H2chain_lattice(a=2.0,t=-1.0):
    # create a simple 1D lattice with vector a1
    lattice = pb.Lattice(a)
    lattice.add_one_sublattice('Ha', 0.0)
    lattice.add_one_sublattice('Hb', a/2)
    lattice.add_one_hopping([0,0], 'Ha', 'Hb', t)
    lattice.add_one_hopping([1,0], 'Hb', 'Ha', t)
    return lattice

H2lattice = H2chain_lattice(aH2,tH2)
H2lattice.plot_brillouin_zone()
plt.show()

H2model = pb.Model(
    H2lattice,
    pb.translational_symmetry()
)
H2model.plot()
plt.show()

H2solver = pb.solver.lapack(H2model)

Gamma = [0, 0]
X = [pi/aH2, 0]
H2bands = H2solver.calc_bands(Gamma, X)

H2bands.plot(point_labels=kpoints_labels)
plt.show()

<div class="rq">

This is again exactly whats is found in textbooks.
<figure>
  <img width = "50%" src="MiscImages/TB/Hoffmann-H2chain-p90.png" alt="H2 chain" />
  <figcaption >(from R. Hoffmann, <i>A Chemist's View of Bonding in Extended Structures</i>, &copy; Wiley 1991)</figcaption>
</figure>

There are two bands now, due to the enlargement of the unit cell. But it does not change anything to the increase of the energy of the band as a function of the *k*, and the nature a state &Psi;(*k*) does not change. 

And the Fermi energy is still 0, with two electrons occupying the lowest band.
</div>

In [ ]:
plot_kdosMP(H2solver, H2lattice, saveFig_path=saveFig_path, FigName="DOS_H1D")

### 1D alternating chain band structure, the Peierls distortion

<div class="intro">

It is the solid-state counterpart of the Jahn-Teller effect. It will occur for systems where a symmetry-lowering deformation will break a degeneracy or quasi-degeneracy of states, provided that it  stabilizes the compound. Let's see how it works on an unsymmetrical chain of hydrogen atoms. Stabilization can be achieved with a distorted hydrogen chain, *i.e.* the hydrogen atoms are in pair. 
</div>

In [ ]:
aH2 = 2*aH
tH2 = tH

def H2Dchain_lattice(a=0.2,t=-1.0):
    # create a simple 1D lattice with vector a1
    lattice = pb.Lattice(a)
    lattice.add_one_sublattice('Ha', 0.0)
    lattice.add_one_sublattice('Hb', 0.8*a/2)
    lattice.add_one_hopping([0,0], 'Ha', 'Hb', 1.2*t)
    lattice.add_one_hopping([1,0], 'Hb', 'Ha', 0.8*t)
    return lattice

H2Dlattice = H2Dchain_lattice(aH2,tH2)
H2Dlattice.plot_brillouin_zone()
plt.show()

H2Dmodel = pb.Model(
    H2Dlattice,
    pb.translational_symmetry()
)
H2Dmodel.plot()
plt.show()

H2Dsolver = pb.solver.lapack(H2Dmodel)

Gamma = [0, 0]
X = [pi/aH2, 0]
H2Dbands = H2Dsolver.calc_bands(Gamma, X)

H2Dbands.plot(point_labels=kpoints_labels)
save_fig("BandStructure_H1D_Peierls", saveFig_path)
plt.show()

In [ ]:
# you can uncomment these lines. But the 1D plot is not very informative :-(

# from math import pi

# H2Dbands = solverSQL.calc_bands(Gamma, X)
# H2Dmodel.lattice.plot_brillouin_zone(decorate=False)
# kpoints_labels = [f'$\Gamma$', 'X']
# SQLbands.plot_kpath(point_labels=kpoints_labels)

<div class="rq">

It is exactly what you can find p. 101 of Hoffmann's book. A stabilization of the system occurs at the Fermi level for any *k*. It is maximal for the half-filled band, *i.e.* at &pi;/(2*a*). The scheme on the right highlights the band gand opening at the Fermi energy.
<figure>
  <img width = "50%" src="MiscImages/TB/Hoffmann-H2distorted_chain-p101.png" alt="distorted H2 chain" />
  <figcaption >(from R. Hoffmann, <i>A Chemist's View of Bonding in Extended Structures</i>, &copy; Wiley 1991)</figcaption>
</figure>
</div>

In [ ]:
plot_kdosMP(H2Dsolver, H2Dlattice, saveFig_path=saveFig_path, FigName="DOS_H1D_Peierls")

## 2D square lattice

### Definition of the primitive lattice model

In [ ]:
import pybinding as pb
import numpy as np
import matplotlib.pyplot as plt

pb.pltutils.use_style() # applies pybinding’s default style settings for matplotlib

def square_lattice(a=0.1, t=-1.):
    '''
    input:
        - a = lattice parameter, in nm (default: 2 angs = 0.2 nm)
        - t = hopping energy, in eV (default : -1 eV)
    '''
    
    # create a simple 2D lattice with vectors a1 & a2
    lattice = pb.Lattice(a1=[a, 0], a2=[0, a])
    lattice.add_sublattices(
        ('A', [0, 0], 0.)  # add an atom called 'A' at position [0, 0] and set its associated AO energy to 0 eV
    )
    lattice.add_hoppings(
        # (Difference of the indices of the source and destination unit cells
        # Name of the sublattice in the source unit cell, Name of the sublattice in the destination unit cell, hopping energy)
        ([0, 1], 'A', 'A', t),
        ([1, 0], 'A', 'A', t)
    )
    return lattice

aSQL = 0.1
tSQL = -1.
SQL = square_lattice(aSQL,tSQL)
SQL.plot()  # plot the lattice that was just constructed
plt.show()      # standard matplotlib show() function

In [ ]:
SQLmodel = pb.Model(
    SQL,
    pb.translational_symmetry()
)
SQLmodel.plot()
plt.show()

<div class="rqT" title="Reminder">

This is a useful representation for checking that the system has been correctly defined using the  add_sublattices` and `add_hoppings` functions, given that:
- The red lines indicate hoppings on periodic boundaries
- The lighter colored circles represent the translations of the unit cell
- The plot only presents the first one in each lattice vector direction
</div>

### Brillouin zone (BZ)

In [ ]:
SQL.plot_brillouin_zone()

<div class="rq">

- The Brillouin zone (BZ) is a square spanned by two vectors **b**<sub>1</sub> and **b**<sub>2</sub>
- Their norm is *&pi;*/*a* (with *a* = lattice parameter)
- This gives the allowed space of *k*: *k*<sub>1</sub>: [0,*&pi;*/*a*] and *k*<sub>2</sub>: [0,*&pi;*/*a*]
- Certain special values of *k* have given names:
    - &Gamma;: [0,0]. It is the BZ center
    - X: [&pi;/*a*,0]) = [0,&pi;/*a*]
    - M: [&pi;/*a*,&pi;/*a*]
</div>
<br>
<div class="intro">

Let's now visualize these special points on the BZ plot. It can be done by calculating the energy levels of the system. 

[`Solver`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.solver.html#pybinding.solver.Solver) has a convenient [`calc_bands()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.solver.html#pybinding.solver.Solver.calc_bands) method which can be used to calculate the band structure of our model

The points $\Gamma, X$ and $M$ are used to draw a path in the reciprocal space of graphene’s Brillouin zone and [`Solver.calc_bands()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.solver.html#pybinding.solver.Solver.calc_bands) calculates the band energy along that path. The return value of the method is a [`Bands`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Bands.html#pybinding.Bands) result object.

All result objects have built-in plotting methods. Aside from the basic [`plot()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Bands.html#pybinding.Bands.plot) seen above, [`Bands`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Bands.html#pybinding.Bands) also has [`plot_kpath()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Bands.html#pybinding.Bands.plot_kpath) which presents the path in reciprocal space. Plots can easily be composed, so to see the path in the context of the Brillouin zone, we can simply plot both:
</div>

In [ ]:
from math import pi
Gamma = [0, 0]
X = [pi/aSQL, 0]
M = [pi/aSQL,pi/aSQL]

solverSQL = pb.solver.lapack(SQLmodel)
SQLbands = solverSQL.calc_bands(Gamma, X, M, Gamma)

SQLmodel.lattice.plot_brillouin_zone(decorate=False)
kpoints_labels = [f'$\Gamma$', 'X', 'M', f'$\Gamma$']
SQLbands.plot_kpath(point_labels=kpoints_labels)
save_fig("BZwithk-HSquareLattice", saveFig_path)
plt.show()

In [ ]:
SQLbands.plot(point_labels=kpoints_labels)
save_fig("BandStructure-HSquareLattice", saveFig_path)
plt.show()

<div class="rq">

M has the highest energy wavefunction and X seems pretty much nonbonding, given its energy. Needless to say that it is again a textbook case:

<figure>
  <img width = "55%" src="MiscImages/TB/Hoffmann-SquareLattice-BandStructure.png" alt="square lattice" />
  <figcaption >(from R. Hoffmann, <i>A Chemist's View of Bonding in Extended Structures</i>, &copy; Wiley 1991)</figcaption> 
</figure>
</div>

In [ ]:
plot_kdosMP(solverSQL, SQL, saveFig_path=saveFig_path, FigName="DOS_HSquareLattice")

### 2 &times; 2 square unitcell

<div class="intro">

A primitive cell is the smallest unit cell of a crystal. For the 2D square lattice, this is the usual 1-atom cell. It’s translated in space to construct a larger system. Sometimes it can be convenient to use a larger unit cell instead, i.e. a supercell consisting of multiple primitive cells.

A 2x2 multiple cell is a 4-atom cell. It can be created by adding three more sublattices to the `Lattice` specification
</div>

In [ ]:
def square_lattice22(a=0.2, t=-1.):
    '''
    input:
        - a = lattice parameter, in nm (default: 2 angs = 0.2 nm)
        - t = hopping energy, in eV (default : -1 eV)
    '''
    
    # create a simple 2D lattice with vectors a1 & a2
    lattice = pb.Lattice(a1=[a, 0], a2=[0, a])
    lattice.add_sublattices(
        ('A', [0, 0], 0.),  # add an atom called 'A' at position [0, 0] and set its associated AO energy to 0 eV
        ('Ax', [a/2, 0], 0.),
        ('Ay', [0, a/2], 0.),
        ('Ad', [a/2, a/2], 0.)
    )
    lattice.add_hoppings(
        # (Difference of the indices of the source and destination unit cells
        # Name of the sublattice in the source unit cell, Name of the sublattice in the destination unit cell, hopping energy)
        # inside the unit cell
        ([0, 0], 'A', 'Ax', t),
        ([0, 0], 'A', 'Ay', t),
        ([0, 0], 'Ax', 'Ad', t),
        ([0, 0], 'Ay', 'Ad', t),
        # between neighbouring unit cells
        ([1, 0], 'Ax', 'A', t),
        ([1, 0], 'Ad', 'Ay', t),
        ([0, 1], 'Ay', 'A', t),
        ([0, 1], 'Ad', 'Ax', t)
    )
    return lattice

aSQL22 = 0.2
tSQL = -1.
SQL22 = square_lattice22(aSQL22,tSQL)
SQL22.plot()  
plt.show()     

In [ ]:
SQL22model = pb.Model(
    SQL22,
    pb.translational_symmetry(a1=True,a2=True)
)
SQL22model.plot()
save_fig("model-HSquareLattice2x2", saveFig_path)
plt.show()

In [ ]:
SQL22.plot_brillouin_zone()

In [ ]:
from math import pi

Gamma = [0, 0]
X = [pi/(aSQL22), 0]
M = [pi/(aSQL22),pi/(aSQL22)]

solverSQL22 = pb.solver.lapack(SQL22model)

SQL22bands = solverSQL22.calc_bands(Gamma, X, M, Gamma)

SQL22model.lattice.plot_brillouin_zone(decorate=False)
kpoints_labels = [f'$\Gamma$', 'X', 'M', f'$\Gamma$']
SQL22bands.plot_kpath(point_labels=kpoints_labels)
save_fig("BZwithk-HSquareLattice2x2", saveFig_path)
plt.show()

SQL22bands.plot(point_labels=kpoints_labels)
plt.savefig("./FiguresCreatedInNotebooks/HSquareLattice2x2.png",dpi=300, transparent=True)
save_fig("BandStructure-HSquareLattice2x2", saveFig_path)
plt.show()


<div class="rq">

You can compare it with the band structure of the primitive square unit cell, reported below. Comment?

<figure>
  <img width = "30%" src="./FiguresCreatedInNotebooks/TB/HSquareLattice-C.png" alt="BSprimitiveSquareLattice" /> 
  <figcaption ><b>Band structure of the primitive square unit cell</b></figcaption> 
</figure>
</div>

In [ ]:
plot_kdosMP(solverSQL22, SQL22, saveFig_path=saveFig_path, FigName="DOS_HSquareLattice2x2")

In [ ]:
eigenvalues = solverSQL22.calc_eigenvalues()
eigenvectors = solverSQL22.eigenvectors

eigenvalues
eigenvectors

eigenvalues.plot()

### 2 &times; 2 rectangle unit cell

<div class="exE">

**1.** Define the 2 &times; 2 unit cell of a rectangular lattice of hydrogen atoms, with:
- *a*<sub>1</sub> = 0.16 nm
- *a*<sub>2</sub> = 0.24 nm
- *t*<sub>1</sub> = 1.2 eV
- *t*<sub>2</sub> = 0.8 eV

**2.** Calculate the band structure: 
- **2a.** visualize the following special *k*-points on the BZ plot
    - &Gamma;: [0, 0]
    - X<sub>1</sub>: [&pi;/*a*<sub>1</sub>, 0]
    - M: [&pi;/*a*<sub>1</sub>, &pi;/*a*<sub>2</sub>]
    - X<sub>2</sub>: [0, &pi;/*a*<sub>2</sub>]
- **2b.** plot the band structure for the same *k*-points

**3.** Plot the DOS

</div>

In [ ]:
# insert your code here



<div class="sol">

Want to see a possible answer? Uncomment the `# %load ./SolutionsToExercises/...` command below, and then run the cell
</div>

In [ ]:
# %load ./SolutionsToExercises/TB/RL22.py

<div class="rq">

You can compare your calculated band structure with the band structure of the 2 &times; 2 square unit cell, reported below. Comment? 

<figure>
  <img width = "40%" src="FiguresCreatedInNotebooks/HSquareLattice2x2-C.png"  alt="RectangleLattice22" />
  <figcaption ><b>Band structure of the 2 &times; 2 square unit cell</b></figcaption> 
</figure>
</div>

## Graphene

<figure>
<img width = "800px" src="MiscImages/Introductions/Graphene_1.webp" alt="Image graphene" />
<figcaption>Image made by R.P. with ChatGPT-4o (OpenAI, 11/2024)</figcaption>
</figure> 

### Definition of the primitive lattice model

<div class="intro">

The honeycomb crystal structure of flat graphene is based on a rhombus primitive unit cell, defined by two vectors **a**<sub>1</sub> and **a**<sub>2</sub>. It contains two atoms only. 

<img width="350" src="./MiscImages/TB/graphene.svg" style="display:block; margin-left:auto; margin-right:auto" id="graphene"/> 
</div>

In [ ]:
import sympy as sp
sp.cos(sp.pi/3)
sp.sin(sp.pi/3)

In [ ]:
import pybinding as pb
import numpy as np
import matplotlib.pyplot as plt

from math import sqrt, pi

def monolayer_graphene(a = 0.24595, eC = 0.0, t = -2.8):
    '''
    input:
    a = 0.24595   # [nm] unit cell length
    eC = 0.0      # [eV] on-site energy
    t = -2.8      # [eV] nearest neighbour hopping
    '''
    d_CC = a/sqrt(3.0)
    lat = pb.Lattice(a1=[a, 0],
                     a2=[a/2, a* sqrt(3)/2])
    lat.add_sublattices(('C1', [0, 0], eC),
                        ('C2', [0, d_CC], eC))
    lat.add_hoppings(
        # inside the main cell
        ([0,  0], 'C1', 'C2', t),
        # between neighboring cells
        ([1, -1], 'C1', 'C2', t),
        ([0, -1], 'C1', 'C2', t)
    )
    return lat

ag = 0.24595 # [nm] unit cell length
tCC = -2.8   # [eV] nearest neighbour hopping
dCCg = ag/sqrt(3.0)
Glattice = monolayer_graphene(ag, tCC)
Glattice.plot()
plt.show()

<div class="intro">

The [`Lattice.add_sublattices()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Lattice.html#pybinding.Lattice.add_sublattices) method creates atoms A and B (blue and orange) at different offsets: $[0, -a_{cc}/2]$ and $[0, a_{cc}/2]$. Once again, the translated cells are given at positions $\vec{R} = n_1 \vec{a}_1 + n_2 \vec{a}_2$, however, this time the lattice vectors are not perpendicular which makes the integer indices $[n_1, n_2]$ slightly more complicate (see the labels in the figure).

The hoppings are defined as follows:

* `([0,  0], 'A', 'B', t)` specifies the hopping inside the main cell, from atom A to B. The main [0,0] cell is never labeled in the figure, but it is always the central cell where the lattice vectors originate.
  
* `([1, -1], 'A', 'B', t)` specifies the hopping between [0, 0] and [1, -1], from A to B. The opposite hopping is added automatically: [-1, 1], from B to A. In the tight-binding matrix representation, the opposite hopping is the Hermitian conjugate of the first one. The lattice specification always requires explicitly mentioning only one half of the hoppings while the other half is automatically added to guarantee hermiticity.
  
* `([0, -1], 'A', 'B', t)` is handled in the very same way.
  
The [`Lattice.plot()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Lattice.html#pybinding.Lattice.plot) method will always faithfully draw any lattice that has been specified. It serves as a handy visual inspection tool.

</div>


### Brillouin zone

<div class="intro">

The method [`Lattice.plot_brillouin_zone()`](http://docs.pybinding.site/page/tutorial/../_api/pybinding.Lattice.html#pybinding.Lattice.plot_brillouin_zone) is another handy tool that does just as its name implies.

</div>

In [ ]:
Glattice.plot_brillouin_zone()

### Band structure calculation

In [ ]:
Gmodel = pb.Model(Glattice,
          pb.translational_symmetry()
)
Gmodel.plot()
save_fig("model_graphene",saveFig_path)
plt.show()

In [ ]:
Gsolver = pb.solver.lapack(Gmodel)

K = [2*pi / (3*sqrt(3)*dCCg), 2*pi / (3*dCCg)]
Gamma = [0, 0]
M = [0, 2*pi / (3*dCCg)]

kpLabels = point_labels=[f'$\Gamma$', 'K', 'M', f'$\Gamma$']

Gmodel.lattice.plot_brillouin_zone(decorate=True)
save_fig("BZ_graphene",saveFig_path)
plt.show()
plt.show()
Gbands = Gsolver.calc_bands(Gamma, K, M, Gamma)

In [ ]:
Gmodel.lattice.plot_brillouin_zone(decorate=False)
Gbands.plot_kpath(point_labels=kpLabels)
save_fig("BZwithk_graphene",saveFig_path)
plt.show()

In [ ]:
Gbands.plot(point_labels=kpLabels)
save_fig("BandStructure_graphene",saveFig_path)
plt.show()

<div class="rqT" title="This is typical of a zero-gap semiconductor">

Graphene is a [zero-gap semiconductor](https://en.wikipedia.org/wiki/Electronic_properties_of_graphene), because its conduction and valence bands meet at Dirac points, *i.e.* the meeting points of two [Dirac cones](https://en.wikipedia.org/wiki/Dirac_cone).

</div>

### DOS plot

<div class="exE" title="DOS plot">

Check that there is no inconsistency between this comment and the DOS plot

</div>

In [ ]:
# insert your code here


<div class="sol">

Want to see a possible answer? Uncomment the `# %load ./SolutionsToExercises/...` command below, and then run the cell
</div>

In [ ]:
# %load ./SolutionsToExercises/TB/DOS_graphene.py

# Useful links and bibliography

[Tight-binding method on wikipedia](https://en.wikipedia.org/wiki/Tight_binding)

[Roald Hoffmann (1987), How Chemistry and Physics Meet in the Solid State, *Angew. Chem. Int. Ed.* **26**:846-878, 10.1002/anie.198708461](https://doi.org/10.1002/anie.198708461)

[Roald Hoffmann (1991), Solids And Surfaces. A Chemist's View of Bonding in Extended Structures, Wiley-VCH](https://www.wiley.com/en-us/Solids+and+Surfaces%3A+A+Chemist%27s+View+of+Bonding+in+Extended+Structures-p-9781119815143)

[Peter Kratzer and Jörg Neugebauer (2019), The Basics of Electronic Structure Theory for Periodic Systems, *Front. Chem.* **7**:106, 10.3389/fchem.2019.00106](https://doi.org/10.3389/fchem.2019.00106)

[Mahesh Datt Bhatt, Heeju Kim, and Gunn Kim (2022), Various defects in graphene: a review, *RSC Adv.* **12**:21520–21547](https://doi.org/10.1039/d2ra01436j)

In [ ]:
vID.end(cwd0)